In [ ]:
import numpy as np
import pandas as pd
import time
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from keras.preprocessing import text, sequence


def load_GloVe_vectors(file, vocab):
    """
    This function will load Global Vectors for words in vocab from the specified GloVe file.
    
    INPUT:
    file  = The path/filename of the file containing GloVe information.
    vocab = The list of words that will be loaded from the GloVe file.
    """
    glove = {}
    with open(file, 'rb') as f:
        for line in f:
            parts = line.split()
            word = parts[0].decode('utf-8')
            if word in vocab:
                vector = np.array(parts[1:], dtype=np.float32)
                glove[word] = vector
    return glove


#Creating Mean Word Embeddings using Mean Embedding Vectorizer class
class W2vVectorizer(object):
    """
    This class is used to provide mean word vectors for review documents. 
    This is done in the transform function which is used to generate mean vectors in model pipelines.
    The class has both fit and transform functions so that it may be used in an sklearn Pipeline.
    """
    
    def __init__(self, model):
        self.w2v = model
        
        #If using GloVe the model is in a dictionary format
        if isinstance(model, dict):
            if len(model) == 0:
                self.dimensions = 0
            else:
                self.dimensions = len(model[next(iter(model))])
        #Otherwise, using gensim keyed vector
        else:
            self.dimensions = model.vector_size
    
    # Need to implement a fit method as required for sklearn Pipeline.
    def fit(self, X, y):
        return self

    def transform(self, X):
        """
        This function generates a w2v vector for a set of tokens. This is done by taking 
        the mean of each token in the review.
        """
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                    or [np.zeros(self.dimensions)], axis=0) for words in X])
    
class KerasTokenizer(object):
    """
    This class is used to fit text and convert text to sequences for use in a Keras NN Model.
    The class has both fit and transform functions so that it may be used in an sklearn Pipeline.
    num_words = max number of words to keep.
    maxlen  = max length of all sequences.
    """
    def __init__(self, num_words=20000, maxlen=100):
        self.tokenizer = text.Tokenizer(num_words=num_words)
        self.maxlen = maxlen
        
    def fit(self, X, y):
        self.tokenizer.fit_on_texts(X)
        return self
        
    def transform(self, X):
        return sequence.pad_sequences(self.tokenizer.texts_to_sequences(X), maxlen=self.maxlen)
    
class KerasModel(object):
    """
    This class is used to fit and transform a keras model for use in an sklearn Pipeline.
    """
    def __init__(self, model, epochs=3, batch_size=32, validation_split=0.1):
        self.model = model
        self.epochs = epochs
        self.batch_size = batch_size
        self.validation_split = validation_split
        
    def set_params(self, epochs=3, batch_size=32, validation_split=0.1):
        self.epochs = epochs
        self.batch_size = batch_size
        self.validation_split = validation_split
        return self
    
    def fit(self, X, y):
        y_dummies = pd.get_dummies(y).values
        self.labels = np.array(pd.get_dummies(y).columns)
        self.model.fit(X, y_dummies, epochs=self.epochs, batch_size=self.batch_size, validation_split=self.validation_split)
        return self
    
    def transform(self, X):
        return X
    
    def predict(self, X):
        y_pred = self.model.predict(X)
        return [self.labels[idx] for idx in y_pred.argmax(axis=1)]
    
    def summary(self):
        self.model.summary()
        
